<a href="https://colab.research.google.com/github/pra17dod/Waste-Segregation/blob/main/model/v2_waste_segregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import re
import random
import seaborn as sns
import cv2 as cv
from PIL import Image
import pickle
import sys
sys.setrecursionlimit(1000000000)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tfhub import load_model_weights

In [ ]:
!git clone https://github.com/garythung/trashnet.git
!unzip /content/trashnet/data/dataset-resized.zip -d /content/

Cloning into 'trashnet'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), done.
Archive:  /content/trashnet/data/dataset-resized.zip
   creating: /content/dataset-resized/
  inflating: /content/dataset-resized/.DS_Store  
   creating: /content/__MACOSX/
   creating: /content/__MACOSX/dataset-resized/
  inflating: /content/__MACOSX/dataset-resized/._.DS_Store  
   creating: /content/dataset-resized/cardboard/
  inflating: /content/dataset-resized/cardboard/cardboard1.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard10.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard100.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard101.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard102.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard103.jpg  
  inflating: /content/dataset-resized/cardboard/cardboard104.jpg  
  inflating: /content/dataset-resi

In [ ]:
os.listdir(os.path.join(os.getcwd(),"dataset-resized"))

['metal', '.DS_Store', 'plastic', 'paper', 'glass', 'trash', 'cardboard']

In [ ]:
## get a path to the folder with images
path = os.path.join(os.getcwd(),"dataset-resized")
path

'/content/dataset-resized'

In [ ]:
print(len(os.listdir('./dataset-resized//')))

137


In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(path, batch_size=16, image_size=(224,224), shuffle=True, label_mode='categorical', validation_split = 0.25, seed = 1, subset = 'training')
valid = tf.keras.preprocessing.image_dataset_from_directory(path, batch_size=16, image_size=(224,224), shuffle=True, label_mode='categorical', validation_split = 0.25, seed = 1, subset = 'validation')

Found 2527 files belonging to 6 classes.
Using 1896 files for training.
Found 2527 files belonging to 6 classes.
Using 631 files for validation.


In [ ]:
model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3))

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
def waste_segment():
    model2 = tf.keras.Sequential()
    model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3), weights='imagenet')
    model2.add(model)
    model2.add(tf.keras.layers.Flatten())
    model2.add(tf.keras.layers.Dense(6, activation='softmax'))
    return model2

In [ ]:
model2 = tf.keras.Sequential()
model2.add(model)
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(Dense(1024, activation = 'relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(Dense(512, activation = 'relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(6, activation='softmax'))

In [ ]:
# model2.load_weights('/content/drive/MyDrive/waste segmentation/waste_weights.h5')

In [ ]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 100352)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)              

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

In [ ]:
model2.fit(x=train, validation_data=valid, epochs = 30)

Epoch 1/30
119/119 [==============================] - 68s 252ms/step - loss: 10.3224 - accuracy: 0.3353 - val_loss: 25.3273 - val_accuracy: 0.2314
Epoch 2/30
119/119 [==============================] - 28s 238ms/step - loss: 1.3384 - accuracy: 0.5114 - val_loss: 1.3156 - val_accuracy: 0.5420
Epoch 3/30
119/119 [==============================] - 29s 242ms/step - loss: 1.1596 - accuracy: 0.5807 - val_loss: 23.3875 - val_accuracy: 0.3439
Epoch 4/30
119/119 [==============================] - 29s 241ms/step - loss: 0.9992 - accuracy: 0.6830 - val_loss: 4.4950 - val_accuracy: 0.4532
Epoch 5/30
119/119 [==============================] - 29s 239ms/step - loss: 1.0587 - accuracy: 0.6327 - val_loss: 117.4376 - val_accuracy: 0.1632
Epoch 6/30
119/119 [==============================] - 29s 241ms/step - loss: 0.9330 - accuracy: 0.7028 - val_loss: 6.5033 - val_accuracy: 0.1632
Epoch 7/30
119/119 [==============================] - 29s 241ms/step - loss: 0.7666 - accuracy: 0.7303 - val_loss: 4.4721 - v

In [ ]:
temp = img_to_array(load_img('/content/dataset-resized/cardboard/cardboard100.jpg', target_size=(224,224))).reshape(1,224,224,3)
np.argmax(model2.predict(temp))

0

In [ ]:
model2.save_weights('waste_weights_heavy.h5')

In [ ]:
!cp waste_weights_heavy.h5 /content/drive/MyDrive/waste\ segmentation/waste_weights_heavy.h5